Recall, there is a lot of categories. We will use LLMs to assign + group 


Zero Shot Classification:
- Taking pre-trained LLM, without further training, gettin model to assign text to category
- We give model with our text and prompt, and may provide it with categories. 
- This is done by leveraging knowledge about the classes' attributes or relationships learned during pre-training

So how can models do this?
- transformers based models are good at understanding what words are semantically related to one-another due to words seen during training
- somewhere in the training data, they may be exposed to similar prompts 


In [43]:
import pandas as pd
books = pd.read_csv("/Users/naomigong/Coding/Book_Recommender/books_cleaned.csv")

In [44]:
books["categories"].value_counts().reset_index().query('count >= 50')

#another method
book_counts = books["categories"].value_counts() #returns pandas series
book_counts[book_counts > 50] #boolean mask, so returns the one that are greater than 50


categories
Fiction                      2269
Juvenile Fiction              456
Biography & Autobiography     348
History                       226
Literary Criticism            138
Philosophy                    129
Religion                      126
Comics & Graphic Novels       125
Drama                          99
Juvenile Nonfiction            81
Science                        62
Poetry                         58
Business & Economics           56
Literary Collections           53
Social Science                 52
Name: count, dtype: int64

In [45]:
#examining the juvenille fiction category
#selecting the rows in which the category is juvenile fiction
books[books["categories"] == "Juvenile Fiction"]
books[books["categories"].str.contains("fiction", case=False, na=False)] #need na = false just in case there are na values


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...
11,9780006483908,0006483909,Jimmy the Hand,Raymond E. Feist;S. M. Stirling,Fantasy fiction,http://books.google.com/books/content?id=hV4-o...,"Jimmy the Hand, boy thief of Krondor, lived in...",2003.0,3.95,368.0,5579.0,Jimmy the Hand,"9780006483908 Jimmy the Hand, boy thief of Kro..."
15,9780006496878,0006496873,Mystical Paths,Susan Howatch,English fiction,http://books.google.com/books/content?id=by4yt...,1968 finds Nicholas Darrow wrestling with pers...,1996.0,4.23,576.0,1023.0,Mystical Paths,9780006496878 1968 finds Nicholas Darrow wrest...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677,9783791329284,3791329286,Joseph Cornell,Joseph Cornell;Alison Baverstock,Juvenile Nonfiction,http://books.google.com/books/content?id=KnO8A...,Provides an introduction to the American artis...,2003.0,4.06,27.0,31.0,Joseph Cornell: Secrets in a Box,9783791329284 Provides an introduction to the ...
5683,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...
5684,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...
5686,9788171565641,8171565646,Aspects of the Novel,E. M. Forster,English fiction,http://books.google.com/books/content?id=qWU9P...,"Forster's lively, informed originality and wit...",2004.0,3.83,141.0,10.0,Aspects of the Novel,"9788171565641 Forster's lively, informed origi..."


In [46]:
#create a mapping
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

books["simplifed_categories"] = books["categories"].map(category_mapping)
books["simplifed_categories"].value_counts()

simplifed_categories
Fiction                  2551
Nonfiction               1029
Children's Fiction        456
Children's Nonfiction      81
Name: count, dtype: int64

It creates a zero-shot classification pipeline

It internally uses the model and tokenizer you specified (facebook/bart-large-mnli)

This pipeline will:

Take an input sentence (e.g., "This book is about science and magic.")

And compare it against a list of labels (e.g., ["sci-fi", "fantasy", "romance"])

The model predicts how likely the input entails each label → and ranks them accordingly.



In [47]:
import transformers 
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model_name = "facebook/bart-large-mnli"
bart_model = AutoModelForSequenceClassification.from_pretrained(model_name)
bart_tokenizer = AutoTokenizer.from_pretrained(model_name)
#for zero shot classification we need to give it the categories
fiction_categories = ["Fiction", "Non Fiction"]
pipe = pipeline("zero-shot-classification", model = model_name)


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
is_torch_greater_or_equal() got an unexpected keyword argument 'accept_dev'